In [ ]:
pip install -r requirements.txt

In [ ]:
import os
import cv2
import numpy as np
import boto3
import albumentations as A
from tqdm import tqdm

# ============================
# 🔐 تنظیمات Arvan و مسیرها
# ============================
access_key = "a8761df0-960e-4dd1-b5f2-ef8ef60823a9"
secret_key = "f502aad1cec94636d4381cadf302a6114df05bf825864e554b82010d6d2441ab"
region_name = "s3.ir-thr-at1.arvanstorage.ir"
bucket_name = "ehsannima"
folder_name = "xray/"
download_dir = "./xray_images"
final_output_dir = "./preprocessed_images"
os.makedirs(download_dir, exist_ok=True)
os.makedirs(final_output_dir, exist_ok=True)

# اتصال به Arvan
s3 = boto3.client(
    "s3",
    endpoint_url=f"https://{region_name}",
    aws_access_key_id=access_key,
    aws_secret_access_key=secret_key,
)

# 📥 دانلود تصاویر
print("📥 Downloading images from Arvan...")
response = s3.list_objects_v2(Bucket=bucket_name, Prefix=folder_name)
for obj in response.get("Contents", []):
    key = obj["Key"]
    if key.endswith((".jpg", ".jpeg", ".png")):
        filename = os.path.basename(key)
        file_path = os.path.join(download_dir, filename)
        s3.download_file(bucket_name, key, file_path)


# ============================
# ⚙️ پیش‌پردازش + Augmentation
# ============================

target_size = (224, 224)
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
num_augmented_per_image = 3

# حذف نواحی سفید
def contour_based_crop(img_gray):
    _, thresh = cv2.threshold(img_gray, 240, 255, cv2.THRESH_BINARY_INV)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not contours:
        return img_gray
    x, y, w, h = cv2.boundingRect(max(contours, key=cv2.contourArea))
    return img_gray[y:y+h, x:x+w]

# resize یا padding برای سایز ثابت
def resize_or_pad(img, size=(224, 224)):
    h, w = img.shape
    target_h, target_w = size
    if h >= target_h and w >= target_w:
        return cv2.resize(img, size)
    delta_h = max(0, target_h - h)
    delta_w = max(0, target_w - w)
    top, bottom = delta_h // 2, delta_h - delta_h // 2
    left, right = delta_w // 2, delta_w - delta_w // 2
    return cv2.copyMakeBorder(img, top, bottom, left, right, cv2.BORDER_CONSTANT, value=0)

# ✅ توالی تغییرات بدون هشدار
transform = A.Compose([
    A.RandomBrightnessContrast(p=0.5),
    A.HorizontalFlip(p=0.5),
    A.Rotate(limit=10, p=0.5),
    A.GaussianBlur(blur_limit=3, p=0.3),
    A.Affine(scale=(0.9, 1.1), translate_percent=0.05, p=0.5),
    A.ElasticTransform(alpha=1.0, sigma=50, alpha_affine=50, p=0.5),
    A.GridDistortion(distort_limit=0.3, p=0.3)
])

print("⚙️ Starting preprocessing and augmentation...")

# 🔁 پردازش تصاویر
for filename in tqdm(os.listdir(download_dir)):
    if not filename.lower().endswith(('.jpg', '.jpeg', '.png')):
        continue

    img_path = os.path.join(download_dir, filename)
    img = cv2.imread(img_path)
    if img is None:
        print(f"⛔ تصویر خراب: {filename}")
        continue

    base_name = os.path.splitext(filename)[0]

    # مرحله 1: تبدیل به grayscale
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # مرحله 2: حذف نوار سفید + resize یا padding
    img_cropped = contour_based_crop(img_gray)
    img_resized = resize_or_pad(img_cropped, size=target_size)

    # مرحله 3: بهبود کنتراست و حذف نویز
    img_clahe = clahe.apply(img_resized)
    img_median_filtered = cv2.medianBlur(img_clahe, 5)  # استفاده از فیلتر میانه برای کاهش نویز
    img_bilateral_filtered = cv2.bilateralFilter(img_median_filtered, 9, 75, 75)  # فیلتر دو طرفه برای حفظ لبه‌ها

    # مرحله 4: Gaussian Blur برای حذف نویز بیشتر
    img_final = cv2.GaussianBlur(img_bilateral_filtered, (3, 3), 0)

    # ذخیره تصویر نهایی
    cv2.imwrite(os.path.join(final_output_dir, f"{base_name}.jpg"), img_final)

    # مرحله 5: ساخت تصاویر افزوده
    for i in range(num_augmented_per_image):
        aug_img = transform(image=img_final)['image']
        aug_name = f"{base_name}_aug{i+1}.jpg"
        cv2.imwrite(os.path.join(final_output_dir, aug_name), aug_img)

print(f"\n✅ همه تصاویر پردازش و در {final_output_dir} ذخیره شدند.")
